<a href="https://colab.research.google.com/github/HeningWang/numpyro_adjective_modelling/blob/main/numpyro_slider_MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import modules and dependencies


In [1]:
!pip install pyro-ppl
!pip install -q numpyro@git+https://github.com/pyro-ppl/numpyro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.5/732.5 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
! git clone https://github.com/HeningWang/numpyro_adjective_modelling.git

Cloning into 'numpyro_adjective_modelling'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 5), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (25/25), 768.23 KiB | 2.40 MiB/s, done.


In [3]:
import os

from IPython.display import set_matplotlib_formats
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

plt.style.use("bmh")
if "NUMPYRO_SPHINXBUILD" in os.environ:
    set_matplotlib_formats("svg")

assert numpyro.__version__.startswith("0.12.1")

In [4]:
import sys
sys.path.append('/content/numpyro_adjective_modelling')

In [5]:
from numpyro_adjective_modelling.core_model import *

Some helper functions:

In [11]:
# Mutate the dataset to include the states of the objects
# ... states are independent variables for models

def extract_states(line):
    states = []
    for i in range(6):
      color = 1 if line[10 + i] == "blue" else 0
      form = 1 if line[16 + i] == "circle" else 0
      new_obj = (i + 1, line[4 + i], color, form)
      states.append(new_obj)
    return jnp.array(states)


# Transform/rescale slider value from range 0 to 100 to 0 to 1
# ... in order to match predicted probability from models

def transformation_data(slider_value, link = None):
    if link == "identity":
      if slider_value > 100:
        slider_value = 100
      elif slider_value < 100:
        slider_value = 0
      transformed_prob = slider_value / 100
    elif link == "logit":
        transformed_prob = 1 / (1 + math.exp(-slider_value))
    return transformed_prob

def link_function(x, link = "identity", param = None):
    if link == "identity":
        return x
    elif link == "logit":
        return np.log(x/(1-x))
    elif link == "rapidlogit":
        # rescale parameter of steepness to be between 0 and 1, and with 0.1 nearly identity
        # param = 50 * param
        return 1 / (1 + torch.exp(param * -(x - 0.5)))

def compute_alpha_beta_concentration(mu, v):
    alpha = mu * v
    beta = (1 - mu) * v
    return alpha, beta

In [18]:
# Check the result of states
df_experiment.loc[0, "states"]

Array([[ 1.,  9.,  1.,  1.],
       [ 2.,  9.,  0.,  1.],
       [ 3., 10.,  0.,  1.],
       [ 4.,  9.,  0.,  0.],
       [ 5.,  9.,  0.,  0.],
       [ 6.,  3.,  0.,  1.]], dtype=float32)

In [12]:
# Import dataset
dataset_url = "/content/numpyro_adjective_modelling/dataset/dataset_slider.csv"
df = pd.read_csv(dataset_url)

# subset data to only include combination dimension_color
df = df[df['combination'] == 'dimension_color']
df.reset_index(inplace=True, drop=True)

# Mutate the dataset to include the states of the objects
df_experiment = df.copy()
df_experiment["states"] = df_experiment.apply(lambda row: extract_states(row), axis=1)
df_experiment.prefer_first_1st = df_experiment.prefer_first_1st.apply(lambda x: transformation_data(x, link = "identity"))

In [ ]:
# split data into train and test set
### change here for whole dataset or subset
#train = split_df(df_experiment, split=True, test_size=0.99, random_state=42)
train = df_experiment
states_train = tuple(train.iloc[:, train.columns.get_loc("states")])
empirical_train = torch.tensor(train.iloc[:, train.columns.get_loc("prefer_first_1st")])

In [ ]:
# define the conditioned model for MCMC
def model_inc_utt_serial_beta(states, data):
    beta = 1
    gamma = pyro.sample("gamma", dist.HalfNormal(5))
    color_semvalue = pyro.sample("color_semvalue", dist.Uniform(0, 1))
    form_semvalue = color_semvalue
    k = pyro.sample("k", dist.Uniform(0, 1))
    cost_weight = 0.5
    wf = 0.5
    #sigma = pyro.sample("sigma", dist.HalfNormal(0.5))
    bias = pyro.sample("bias", dist.HalfNormal(5)) + 1
    stepness = pyro.sample("stepness", dist.HalfNormal(0.5))
    v = pyro.sample("v", dist.Uniform(1e-5,5))
    for i in pyro.plate("data_loop", len(data)):
        model_prob = get_results(inc_speaker_utt_production(states[i], beta, gamma, color_semvalue, form_semvalue,wf,k,cost_weight, bias))["probs"][0]
        slider_predict = link_function(model_prob, link = "rapidlogit", param = stepness)
        slider_predict = torch.clamp(slider_predict, 1e-5, 1-1e-5)
        obs = torch.clamp(data[i], 1e-5, 1-1e-5)
        alpha, beta = compute_alpha_beta_concentration(slider_predict, v)
        pyro.sample("obs_{}".format(i), dist.Beta(alpha,beta), obs=obs) # use this for inference
        #pyro.sample("obs_{}".format(i), dist.Beta(alpha,beta)) # use this for prior predictive


In [ ]:
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

In [ ]:
    # define the MCMC kernel and the number of samples
    kernel = NUTS(model_inc_utt_serial_beta, target_accept_prob=0.5)
    mcmc_inc = MCMC(kernel, num_warmup=10,num_samples=10)
    mcmc_inc.run(states_train, empirical_train)

    # print the summary of the posterior distribution
    mcmc_inc.summary()
    mcmc_inc.diagnostics()

    # Get the MCMC samples and convert to a DataFrame
    posterior_inc = mcmc_inc.get_samples()
    df_inc = pd.DataFrame(posterior_inc)

    # Save the DataFrame to a CSV file
    df_inc.to_csv('posterior_inc_utt_slider.csv', index=False)

AttributeError: ignored